In [1]:
from visiprog.data import *
from visiprog.metriclearning import *
from visiprog.evaluate import *
%matplotlib inline
import numpy as np
from visiprog.metric_learn.fda_visiprog import LFDA_VISIPROG
from metric_learn import Covariance

from IPython.display import Image, display
from sklearn.model_selection import train_test_split
import pandas as pd

from sklearn.cluster import KMeans

In [2]:
raw_feature = np.genfromtxt('visiprog/data/curetaliasfree.csv',delimiter=",")
label_material = read_material_label()
groups_VSP, material = read_VSP_group(pappas_only=False)

print(raw_feature.shape)
print(label_material.shape)

(5245, 82)
(5245,)


In [3]:
label_visiprog = get_VSP_label(groups_VSP, raw_feature.shape[0])

print(np.max(label_visiprog))

200.0


In [4]:
model = Covariance()
model.fit(raw_feature)
X_STSIMM = model.transform(raw_feature)
# print(X_STSIMM)

In [6]:
accuracy_STSIMM = []
accuracy_mat = []
accuracy_vsp_original = []
accuracy_vsp = []

N_material = 61
N_material_train = 50

for fold in range(10):
    print("Trial {}".format(fold))

    # randomly select N_material_train material for training the metric
    # and test on the rest
    train_select = np.zeros_like(label_material).astype(bool)

    idx_train = np.array([])
    for i in range(N_material_train):
        train_class = np.random.randint(0, N_material)
        train_select |= (label_material == train_class)

    idx_train = np.where(train_select == True)[0]
    idx_test = np.where(train_select == False)[0]

    idx_train = idx_train.astype(int)
    idx_test = idx_test.astype(int)

    print("Training shape {} and test shape {}".format(len(idx_train), len(idx_test)))

    X_train = raw_feature[idx_train,:]
    X_STSIMM_train = X_STSIMM[idx_train,:]
    Y_VSP_train = label_visiprog[idx_train]
    Y_mat_train = label_material[idx_train]

    X_test = raw_feature[idx_test,:]
    X_STSIMM_test = X_STSIMM[idx_test,:]
    Y_VSP_test = label_visiprog[idx_test]
    Y_mat_test = label_material[idx_test]

    res_STSIMM = leave_one_sample_out(X_STSIMM_test, Y_mat_test)
    accuracy_STSIMM.append(res_STSIMM['accuracy'])

    model = LFDA()
    try:
        model.fit(X_train, Y_mat_train)
        X_mat_test = model.transform(X_test)
        res = leave_one_sample_out(X_mat_test, Y_mat_test)
        accuracy_mat.append(res['accuracy'])
    except Exception as e:
        acuracy_mat.append(np.nan)

    model_vsp_original = LFDA()
    try:
        # remove the unlabeled data
        idx_outlier = np.where(Y_VSP_train == 0)[0]
        X_train_without_outlier = X_train[~idx_outlier,:]
        Y_VSP_train_without_outlier = Y_VSP_train[~idx_outlier]
        
        model_vsp_original.fit(X_train_without_outlier, Y_VSP_train_without_outlier)
        X_vsp_original_test = model_vsp_original.transform(X_test)
        res_vsp_original = leave_one_sample_out(X_vsp_original_test, Y_mat_test)
        accuracy_vsp_original.append(res_vsp_original['accuracy'])
    except Exception as e:
        accuracy_vsp_original.append(np.nan)

    model_vsp = LFDA_VISIPROG()
    try:
        model_vsp.fit(X_train, Y_VSP_train)
        X_vsp_test = model_vsp.transform(X_test)
        res_vsp = leave_one_sample_out(X_vsp_test, Y_mat_test)
        accuracy_vsp.append(res_vsp['accuracy'])
    except Exception as e:
        accuracy_vsp.append(np.nan)

Trial 0
Training shape 2699 and test shape 2546
[24  1  1 ..., 52 57 57]
[ 1  1  1 ..., 57 57 57]
Trial 1
Training shape 2700 and test shape 2545
[ 4  4  4 ..., 59 59 58]
[ 4  4  4 ..., 59 59 59]
Trial 2
Training shape 2693 and test shape 2552
[ 5  1  1 ..., 60 60 60]
[ 1  1  1 ..., 60 60 60]
Trial 3
Training shape 2735 and test shape 2510
[21  0  0 ..., 59 59 59]
[ 0  0  0 ..., 59 59 59]
Trial 4
Training shape 3007 and test shape 2238
[21  0  0 ..., 59 59 58]
[ 0  0  0 ..., 59 59 59]
Trial 5
Training shape 2640 and test shape 2605
[21  0  0 ..., 60 60 60]
[ 0  0  0 ..., 60 60 60]
Trial 6
Training shape 3100 and test shape 2145
[21  0  0 ..., 60 60 60]
[ 0  0  0 ..., 60 60 60]
Trial 7
Training shape 3154 and test shape 2091
[51  2  2 ..., 55 55 55]
[ 2  2  2 ..., 55 55 55]
Trial 8
Training shape 2888 and test shape 2357
[24  1  1 ..., 59 59 59]
[ 1  1  1 ..., 59 59 59]
Trial 9
Training shape 3157 and test shape 2088
[ 4  4  4 ..., 60 60 60]
[ 4  4  4 ..., 60 60 60]


In [7]:
df = pd.DataFrame({'visiprog':accuracy_vsp, 'material':accuracy_mat, 'STSIMM':accuracy_STSIMM, \
       'visiprog_original':accuracy_vsp_original}, columns=['STSIMM','material','visiprog_original','visiprog'])

df.loc['average'] = df.mean()

display(df)
# display(df.mean())

STSIMM  material  visiprog_original  visiprog
0        0.921838  0.994108           0.981540  0.994501
1        0.936346  0.993320           0.987426  0.994499
2        0.896944  0.993730           0.977273  0.994122
3        0.938247  0.988048           0.979681  0.990837
4        0.940125  0.992851           0.972297  0.993298
5        0.931670  0.983109           0.961996  0.985029
6        0.915618  0.990676           0.976224  0.989277
7        0.936394  0.998087           0.988522  0.997609
8        0.921086  0.991939           0.983029  0.994060
9        0.913314  0.992816           0.975575  0.990421
average  0.925158  0.991869           0.978356  0.992365

In [ ]:
print(model.transformer())